In [ ]:
#환경설정
from bs4 import BeautifulSoup
from selenium import webdriver
import time
import sys
import math
import os
import urllib
import urllib.request

## 스크롤다운 함수
def scroll_down(driver):
    driver.execute_script("window.scrollTo(0,document.body.scrollHeight);")
    time.sleep(1)

f_dir = input('사진을 저장할 폴더를 지정하세요(예: C:\\study\\python\\notebook\\data\\) : ')
do_txt = input('크롤링할 키워드는 무엇입니까?: ')
cnt = int(input('크롤링할 건수는 몇 건입니까?'))
page_cnt=math.ceil(cnt/10)

#파일을 저장할 폴더 생성
now = time.localtime()
s = '%04d-%02d-%02d-%02d-%02d-%02d' %(now.tm_year, now.tm_mon, now.tm_mday, now.tm_hour, now.tm_min, now.tm_sec)
s_short = '%04d-%02d-%02d' %(now.tm_year, now.tm_mon, now.tm_mday)

#입력받은 경로+현재 날짜_사진저장이라는 폴더 생성
os.makedirs(f_dir+s+'-'+do_txt)
f_result_dir = f_dir+s+'-'+do_txt
#폴더명 출력
print(f_result_dir)

#크롬 드라이버를 사용하여 웹 브라우저 실행
s_time = time.time()
path = "C:/study/python/datadown/chromedriver.exe"
driver = webdriver.Chrome(path)
driver.get('https://korean.visitkorea.or.kr')
time.sleep(2)  # 페이지가 모두 열릴 때 까지 2초 기다립니다.
driver.maximize_window()

#코로나 alert 끄기
try :
    driver.find_element_by_xpath('//*[@id="chkForm01"]').click()
except :
    print("코로나 창이 없습니다")

#키워드 검색
element = driver.find_element_by_id('inp_search')

element.send_keys(do_txt)
element.send_keys('\n')

html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')
print("=" *80)
print("크롤링 할 키워드 : ", do_txt)
print("크롤링 할 총 페이지 번호: ",page_cnt)
print("=" *80)

#스크롤 다운 실행    
# scroll_down(driver) 

#본문의 사진 정보를 가져와 지정된 폴더에 저장하기
no=1
file_no=1
folder_no=1
ftitle=[]
img_src1=[]

for z in range(1,page_cnt+1):
    print("%s 페이지 내용 수집 시작합니다 =======================" %z)
    for y in range(1, 11):
        if no > cnt:
            break
        #각 게시글 제목 누르기
        try:
            driver.find_element_by_xpath('//*[@id="listBody"]/ul/li[%s]/div[2]/div[1]/a' %y).click()
        except:
            continue
        time.sleep(2)

        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')

        try:
            # try:
            #     for
            #     html=driver.page_source
            #     soup=BeautifulSoup(html,'html.parser')
            try:
                html=driver.page_source
                soup=BeautifulSoup(html,'html.parser')

                title = soup.find('h2',id='topTitle').get_text()
                ftitle.append(title)
                print(title)
                img_c=soup.find('div','box_txtPhoto').find_all('img')
                img_src2=[]
                for i in img_c:
                    img_src=i['src'] #이미지 태그의 src 속성값(파일경로및파일명) 가져오기
                    img_src2.append(img_src)
                    
                img_src1.append(img_src2)

            except:
                print('이미지 게시글이 아닙니다.')

            try:
                title=soup.find('div',id='contents').find('h2').get_text()
                for x in range(1,5):
                    driver.find_element_by_xpath("""//*[@id="contents"]/div[2]/div[6]/div[1]/div[1]/div/ul/li[{}]/a""".format(x)).click()
                    time.sleep(2)
                    html=driver.page_source
                    soup=BeautifulSoup(html,'html.parser')
                    subtitle = soup.find('ul','swiper-wrapper cosList').find_all('div','tit')[x-1].get_text()
                    ftitle.append(title+'-'+subtitle)
                    tag = soup.find('div',"cos_cont active")
                    tag2 = tag.find('div','info_area pc').find_all('a')
                    
                    img_src2=[] #폴더별 리스트 생성

                    for i in tag2:
                        style = i['style'].split('(')[1].split(')')[0]
                        img_src2.append(style)

                    img_src1.append(img_src2)
            except:
                print('코스 게시글이 아닙니다.')

            driver.back()
            time.sleep(2)

            no +=1
        except:
            driver.back()
            continue
    if z>page_cnt+1:
        break
    z+=1

    if (z % 5 == 1):
            driver.find_element_by_link_text('다음').click()
    else :
            driver.find_element_by_link_text("%s"%z).click() # 다음 페이지번호 클릭    
    time.sleep(2)


for k in range(0,len(ftitle)):
    print("폴더내 %s번째 폴더 생성" %folder_no)
    file_no=1
    folder_no += 1
    os.makedirs(f_dir+s+'-'+do_txt+'\\'+s_short+ftitle[k])
    os.chdir(f_dir+s+'-'+do_txt+'\\'+s_short+ftitle[k])
    for i in range(0,len(img_src2)):
        try:
            urllib.request.urlretrieve(img_src1[k][i],str(file_no)+'.jpg')
        except:
            continue
        print("%s 번째 이미지 저장중입니다========" %file_no)
        file_no += 1
        time.sleep(0.5)

#요약정보 출력
e_time=time.time()
t_time=e_time-s_time

print("=" *70)
print("총 소요시간은 %s 초 입니다 " %round(t_time,1))
print("총 저장 건수는 %s 건 입니다 " %file_no)
print("파일 저장 경로: %s 입니다" %f_result_dir)
print("=" *70)
 
driver.close( )

사진을 저장할 폴더를 지정하세요(예: C:\study\python\notebook\data\) : C:\study\python\notebook\data\
크롤링할 키워드는 무엇입니까?: 정동진 해변
크롤링할 건수는 몇 건입니까?30
C:\study\python\notebook\data\2021-04-15-17-38-22-정동진 해변
크롤링 할 키워드 :  정동진 해변
크롤링 할 총 페이지 번호:  3
1 페이지 내용 수집 시작합니다 =======================
정동진해변
이미지 게시글이 아닙니다.
코스 게시글이 아닙니다.
정동진 모래시계 공원(2016년 선정 열린관광지 )
코스 게시글이 아닙니다.
기암·항구·해변이 멋진 여름 바다로 떠나다, 주문진 아들바위
코스 게시글이 아닙니다.
이미지 게시글이 아닙니다.
순천만에서 해돋이를 보며 추억을 낚다, 순천 화포해변
코스 게시글이 아닙니다.
[강릉] 정동진 바다가 그립다면, 호텔탑스빌
코스 게시글이 아닙니다.
정동진 레일바이크
이미지 게시글이 아닙니다.
코스 게시글이 아닙니다.
마음 시린 날 커피 한잔의 위로, 강릉 사천진해변
코스 게시글이 아닙니다.
정동진역
이미지 게시글이 아닙니다.
코스 게시글이 아닙니다.
2 페이지 내용 수집 시작합니다 =======================
해안 비경 품은 환상의 드라이브 코스, 강릉 헌화로
코스 게시글이 아닙니다.
[▶LIVE] 전국 해맞이 명소, 일출라이브!새해 첫 해 배달해드립니다! 안방 1열에서 만나보세요.
코스 게시글이 아닙니다.
이미지 게시글이 아닙니다.
자연을 상영하는 기차, 바다열차 & 정선아리랑열차
코스 게시글이 아닙니다.
한결같은 그 모습 그대로의 강원도
코스 게시글이 아닙니다.
태초의 동해가 밀려왔다, 정동심곡바다부채길
코스 게시글이 아닙니다.
미리 가보는 2018평창 관광로드 04, 숲, 바다, 예술이 어우러진 강릉 여름 여행
코스 게시글이 아닙니다.
[강원] 여행에 휴식을 더하는 강원도의 방
코스 게시글이 아닙니다.
미리 가보는 201